# Семинар 24. Генераторы, менеджеры контекста

## Генераторы

- `range`
- `enumerate`
- `map`
- `filter`
- `zip`
- generator expressions, напр. `(x**2 for x in range(10))`


Полезные ссылки:

- [realpython/generators](https://realpython.com/introduction-to-python-generators/)
- [realpython/itertools](https://realpython.com/python-itertools/)

In [1]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [3]:
gen = (x**2 for x in range(10))

In [9]:
next(gen)

25

In [10]:
for i, e in enumerate([1, 2, 3, 4]):
    print(i, e)

0 1
1 2
2 3
3 4


In [11]:
gen = enumerate([1, 2, 3, 4])

In [17]:
next(gen)

StopIteration: 

In [19]:
gen = map(lambda x: x + 2, [1, 2, 3])

In [23]:
next(gen)

StopIteration: 

In [24]:
gen = filter(lambda x: x%2, range(10))

In [30]:
next(gen)

StopIteration: 

In [55]:
gen = zip([1, 2], 'abc', (3.4, 5.6, 7.8))

In [56]:
next(gen)

(1, 'a', 3.4)

### `yield`

In [42]:
def mygenerator():
    print('Первый вызов')
    yield 1

    print('Второй вызов')
    yield 2

    print('Последний вызов')
    yield 3

In [43]:
for i in mygenerator():
    print(i)

Первый вызов
1
Второй вызов
2
Последний вызов
3


In [44]:
list(mygenerator())

Первый вызов
Второй вызов
Последний вызов


[1, 2, 3]

In [45]:
gen = mygenerator()
next(gen)

Первый вызов


1

In [46]:
next(gen)

Второй вызов


2

In [47]:
next(gen)

Последний вызов


3

### аналог `range`

In [48]:
def sign(x):
    return 1 if x >= 0 else -1

def myrange(start, stop, step=1):
    s = sign(stop - start)
    if s != sign(step):
        return None
    i = start
    while s * i < s * stop:
        yield i
        i += step

In [53]:
list(myrange(-10, 10))

[-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Задание

- создать аналог генератора `enumerate`

- создать генератор чисел Фибоначчи

- создать аналог генераторов из модуля `itertools`:
    - `cycle(iterable)`
    - `repeat(obj, times=1)`
    - `product(*iterables)`


In [57]:
def myenumerate(iterable):
    return zip(range(len(iterable)), iterable)

In [68]:
def myenumerate2(iterable):
    it = iter(iterable)
    i = 0
    while True:
        try:
            yield i, next(it)
            i += 1
        except StopIteration:
            return None

In [69]:
for i, e in myenumerate2([1, 2, 3, 4]):
    print(i, e)

0 1
1 2
2 3
3 4


In [54]:
from itertools import product

list(product([1, 2, 3], 'ab'))

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]

## Менеджер контекста

```
with context as ctx:
    # do something using ctx
    
# here ctx will be automatically closed
```


Пример:

```
with open('file.txt', 'rt') as f:
    print(f.read())
```


In [70]:
import numpy as np

In [71]:
arr = np.array([0.123456, 1.234])

In [72]:
print(arr)

[0.123456 1.234   ]


In [ ]:
np.set_printoptions()

In [73]:
with np.printoptions(precision=2):
    print(arr)

[0.12 1.23]


In [74]:
print(arr)

[0.123456 1.234   ]


In [75]:
import sqlite3

In [76]:
# менеджер контекста для соединения с базой данных sqlite
class DBConn:
 
    def __init__(self, db_name):
        self.db_name = db_name
    
    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.close()
        if exc_val:
            raise

fname = 'test.db'

In [77]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE Persons (PersonID int, LastName varchar(255), FirstName varchar(255))')
    conn.commit()

In [78]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute("INSERT INTO Persons VALUES (0, 'Ivan', 'Petrov')")
    conn.commit()

In [79]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Persons')
    for rec in cursor.fetchall():
        print(rec)

(0, 'Ivan', 'Petrov')


In [80]:
with sqlite3.connect(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Persons')
    for rec in cursor.fetchall():
        print(rec)

(0, 'Ivan', 'Petrov')


## `contextlib`

- `@contextmanager`
- `contextlib.suppress(*exceptions)`

Полезные ссылки:

- [contextlib](https://docs.python.org/3/library/contextlib.html)

In [86]:
from contextlib import contextmanager

In [57]:
@contextmanager
def open_db(fname):
    try:
        conn = sqlite3.connect(fname)
        yield conn
    finally:
        conn.close()

In [81]:
import os
from contextlib import suppress

In [82]:
os.remove('somefile.tmp')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'somefile.tmp'

In [83]:
with suppress(FileNotFoundError):
    os.remove('somefile.tmp')

## Задание

- создать менеджер контекста, позволяющий удобно выполнить какие-либо действия в заданной папке (например, создать файл)

- создать менеджер контекста, позволяющий перенаправить `stdout` в другой поток (например, в файл)
    - необходимо временно связать ссылку `sys.stdout` с другим потоком


In [ ]:
os.chdir()

In [87]:
@contextmanager
def set_folder(folder):
    cwd = os.getcwd()
    try:
        os.chdir(folder)
        yield
    finally:
        os.chdir(cwd)

In [88]:
with set_folder('C:\\Users\\stasb'):
    with open('file.txt', 'w') as f:
        f.write('something')

In [93]:
with set_folder('C:\\Users\\stasb'):
    print(os.getcwd())
    with open('file.txt', 'r') as f:
        print(f.read())

C:\Users\stasb
something


In [91]:
with open('file.txt', 'r') as f:
       print(f.read())

FileNotFoundError: [Errno 2] No such file or directory: 'file.txt'

In [94]:
os.getcwd()

'C:\\Users\\stasb\\YandexDisk\\py\\PythonCourse2021'

In [ ]:
print()

In [95]:
import sys

@contextmanager
def redirect_stdout(stream):
    old = sys.stdout
    sys.stdout = stream
    try:
        yield
    finally:
        sys.stdout = old

In [96]:
import io
stream = io.StringIO()
with redirect_stdout(stream):
    print("Hello, World!")

In [97]:
stream.getvalue()

'Hello, World!\n'